# Coregister MR and head coordinate systems

The basic workflow for `mne-python`-users is described [in the MNE cookbook](http://martinos.org/mne/stable/manual/cookbook.html#overview).

## How many transformations do I need?

The objective of this stage of analysis is to link the `head` and `MRI` coordinate systems. Note that the `head`-to-`MEG device` transformation is a separate issue (obtained using a fit to the HPI coil signals). _We only need one `head`-to-`MRI` per recording session ("study" in StormDB)._

## `mne.gui.coregister`

Make sure you have a recent version of `mne-python` available (_e.g._, by including `use mne` in your `bash` startup file. Open a Terminal window, set the `SUBJECTS_DIR` environment variable to the current project (_e.g._, by `set_mindlabproj` followed by a `gotoproj`), and write

```bash
mne coreg
```

Please see [these slides](http://www.slideshare.net/mne-python/mnepython-coregistration) for details on how to use the GUI.

## Combined MEG-EEG measurement

When an EEG cap is placed on the head, it is not recommended to even try to digitise extra points on the head: these would be _very_ likely to end up not touching the scalp under the cap and lead to a poor coregistration. On the other hand, EEG electrodes are digitised such that the _scalp point_ in the middle of the ring is chosen as the location. EEG electrode locations can thus be used as "extra" points in the coregistration.

At present (Feb 2017), the `mne.gui.coregistration`-tool does not support using EEG locations as head shape points. In this notebook we demonstrate how to modify the header ('`info`') structure of a recording to make the GUI treat EEG locations as head shape points.

### Create an empty raw file for each study date

We'll simply take a single raw MEG recording per subject, per study date, and write them out to a `trans`-folder in the project's `scratch`.

In [ ]:
from mne.io import read_raw_fif, RawArray, read_info
from mne.io.constants import FIFF
from stormdb.access import Query
from stormdb.base import mkdir_p  # mkdir -p
import os.path as op
import numpy as np

In [ ]:
proj_name = 'MINDLAB2015_MEG-Sememene'
trans_dir = op.join('/projects', proj_name, 'scratch', 'trans')
mkdir_p(trans_dir)  # this must exist

In [ ]:
qy = Query(proj_name)
included_subjects = qy.get_subjects()

The following loop over included subjects will

* find all available MEG-studies, then for each study
* read the digitised points of one recording
* "re-label" EEG points as "extra" head shape points (original data not touched)
* save an empty raw-file with the modified digitisation information
    * file name: `trans_dir`/`subject_studydate_coreg.fif`

In [ ]:
for sub in included_subjects:
    studies = qy.get_studies(sub, modality='MEG')
    for study in studies:
        study_date = study[:8]  # cut out the '_000000'
        output_fname = '_'.join([sub, study_date, 'coreg.fif'])

        series = qy.filter_series('*', subjects=sub,
                                  study_date_range=study_date)
        # any file from this study will do, use the first one
        raw_fname = op.join(series[0]['path'], series[0]['files'][0])
        
        info = read_info(raw_fname)
        # do an in-place replacement, consider making a copy and
        # adding the EEG points as EXTRA points to the end?
        for digpoint in info['dig']:
            if digpoint['kind'] == FIFF.FIFFV_POINT_EEG:
                digpoint['kind'] = FIFF.FIFFV_POINT_EXTRA
        raw = RawArray(np.empty((info['nchan'], 1)), info)
        raw.save(op.join(trans_dir, output_fname), overwrite=True) 

### What now?

Use the newly-created empty raw-files as the "Head Shape Source (Raw/Epochs/Evoked)" in `mne.gui.coregistration`. Remember that you only need one transformation per recording session (_i.e._, study).